<a href="https://colab.research.google.com/github/KORALLLL/MTUCI_EMNIST/blob/Nastya/lenet/ReschOfBNL%26DPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameters

train dataset: MNIST + EMNIST letters

test dataset: dataset of DreamTeam

training sample size: 36960 * 2

validation sample size: 30800

testing sample size: 16784

loss finction: Cross Entropy Loss

optimizer: RMSprop

learning rate: 0.001

batch size: 2640

epoch number: 500

#Module importation

In [2]:
import torch
import torchvision.datasets
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as bar
import pickle
import numpy as np
import scipy.stats as stats

#Git cloning

In [3]:
!git clone https://github.com/KORALLLL/MTUCI_EMNIST.git

Cloning into 'MTUCI_EMNIST'...
remote: Enumerating objects: 36511, done.
remote: Counting objects: 100% (12871/12871), done.
remote: Compressing objects: 100% (12702/12702), done.
remote: Total 36511 (delta 192), reused 12805 (delta 160), pack-reused 23640
Receiving objects: 100% (36511/36511), 128.97 MiB | 29.61 MiB/s, done.
Resolving deltas: 100% (477/477), done.


#Dataset preparation

In [4]:
emnist_train = torchvision.datasets.EMNIST('./', split='letters', download = True, train = True)
mnist_train = torchvision.datasets.EMNIST('./', split='mnist', download = True, train = True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

emnist_train_data = emnist_train.data
mnist_train_data = mnist_train.data
train_data = torch.cat([emnist_train_data[:24960], mnist_train_data[:12000]], dim=0).float().unsqueeze(1).to(device) / 255

temp_emnist_train_labels = emnist_train.targets[:24960]
emnist_train_labels = []

for i in range(len(temp_emnist_train_labels)):
  if temp_emnist_train_labels[i]==15:
    emnist_train_labels.append(torch.tensor(0))
  elif temp_emnist_train_labels[i]>15:
    emnist_train_labels.append(temp_emnist_train_labels[i]+8)
  else:
    emnist_train_labels.append(temp_emnist_train_labels[i]+9)

mnist_train_labels = mnist_train.targets
train_labels = torch.cat([torch.stack(emnist_train_labels[:24960]), mnist_train_labels[:12000]], dim=0).to(device)

file = open('MTUCI_EMNIST/dataset.pkl', 'rb')
test_dataset = pickle.load(file)
file.close()

test_data = test_dataset['data'].numpy()
test_data = np.flip(test_data, axis = 3)
test_data = np.rot90(test_data, k=1, axes=(2,3))
test_data = 1 - test_data
test_data = torch.from_numpy(test_data).float().to(device)
test_labels = test_dataset['targets'].to(device)

100%|██████████| 561753746/561753746 [00:07<00:00, 74118169.44it/s] 


Extracting ./EMNIST/raw/gzip.zip to ./EMNIST/raw


#First net

In [4]:
class Lenet1(torch.nn.Module):
  def __init__(self):
    super(Lenet1, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
    self.act1 = torch.nn.ReLU6()
    self.conv2 = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, padding=1)
    self.act2 = torch.nn.ReLU6()
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=6, kernel_size=3, padding=1)
    self.act3 = torch.nn.ReLU6()
    self.conv4 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
    self.act4 = torch.nn.ReLU6()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act5 = torch.nn.ReLU6()
    self.conv5 = torch.nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=0)
    self.act6 = torch.nn.ReLU6()
    self.conv6 = torch.nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, padding=1)
    self.act7 = torch.nn.ReLU6()
    self.conv7 = torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3, padding=0)
    self.act8 = torch.nn.ReLU6()
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
    self.act9 = torch.nn.ReLU6()
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act10 = torch.nn.ReLU6()
    self.fc1 = torch.nn.Linear(16*5*5, 120)
    self.act11 = torch.nn.ReLU6()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act12 = torch.nn.ReLU6()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    skipConn1 = x

    x = self.conv2(x)
    x = self.act2(x)
    x = torch.add(skipConn1, x)

    x = self.conv3(x)
    x = self.act3(x)
    skipConn2 = x

    x = self.conv4(x)
    x = self.act4(x)
    x = torch.add(skipConn2, x)

    x = self.pool1(x)
    x = self.act5(x)
    x = self.conv5(x)
    x = self.act6(x)
    skipConn3 = x
    x = self.conv6(x)
    x = self.act7(x)
    x = torch.add(x, skipConn3)
    x = self.conv7(x)
    x = self.act8(x)
    skipConn4 = x

    x = self.conv8(x)
    x = self.act9(x)
    x = torch.add(skipConn4, x)

    x = self.pool2(x)
    x = self.act10(x)
    x = x.view(x.size(0), x.size(1)*x.size(2)*x.size(3))
    x = self.fc1(x)
    x = self.act11(x)
    x = self.fc2(x)
    x = self.act12(x)
    x = self.fc3(x)
    return x

In [5]:
batch_size = 2640
loss = torch.nn.CrossEntropyLoss()
lenet = Lenet1().to(device)
optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
test_accuracy_history = []

for epoch in bar(range(200), desc = 'learning'):
  order = np.random.permutation(len(train_data))

  for start_index in range(0, len(train_data), batch_size):
    optimizer.zero_grad()

    batch_indexes = order[start_index:start_index+batch_size]

    data_batch = train_data[batch_indexes].to(device)
    label_batch = train_labels[batch_indexes].to(device)

    preds = lenet.forward(data_batch)

    loss_val = loss(preds, label_batch)
    loss_val.backward()

    optimizer.step()

  with torch.no_grad():
    test_preds = lenet.forward(test_data)
    test_accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
    test_accuracy_history.append(test_accuracy)

<ipython-input-5-7d14ee654538>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in bar(range(200), desc = 'learning'):


learning:   0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
import plotly.graph_objects as go

In [11]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history, mode ="lines+markers"))
fig.update_xaxes(range=[20,200])
fig.update_yaxes(range=[0.6, 0.8])
fig.update_layout(title="accuracy default model", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#120

# Second net

In [12]:
class Lenet2(torch.nn.Module):
  def __init__(self):
    super(Lenet2, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
    self.bn1 = torch.nn.BatchNorm2d(4)
    self.act1 = torch.nn.ReLU6()
    self.conv2 = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, padding=1)
    self.bn2 = torch.nn.BatchNorm2d(4)
    self.act2 = torch.nn.ReLU6()
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=6, kernel_size=3, padding=1)
    self.bn3 = torch.nn.BatchNorm2d(6)
    self.act3 = torch.nn.ReLU6()
    self.conv4 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
    self.bn4 = torch.nn.BatchNorm2d(6)
    self.act4 = torch.nn.ReLU6()
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act5 = torch.nn.ReLU6()
    self.conv5 = torch.nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=0)
    self.bn5 = torch.nn.BatchNorm2d(12)
    self.act6 = torch.nn.ReLU6()
    self.conv6 = torch.nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, padding=1)
    self.bn6 = torch.nn.BatchNorm2d(12)
    self.act7 = torch.nn.ReLU6()
    self.conv7 = torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3, padding=0)
    self.bn7 = torch.nn.BatchNorm2d(16)
    self.act8 = torch.nn.ReLU6()
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
    self.bn8 = torch.nn.BatchNorm2d(16)
    self.act9 = torch.nn.ReLU6()
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act10 = torch.nn.ReLU6()
    self.fc1 = torch.nn.Linear(16*5*5, 120)
    self.act11 = torch.nn.ReLU6()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act12 = torch.nn.ReLU6()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act1(x)
    skipConn1 = x

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.act2(x)
    x = torch.add(skipConn1, x)

    x = self.conv3(x)
    x = self.bn3(x)
    x = self.act3(x)
    skipConn2 = x

    x = self.conv4(x)
    x = self.bn4(x)
    x = self.act4(x)
    x = torch.add(skipConn2, x)

    x = self.pool1(x)
    x = self.act5(x)
    x = self.conv5(x)
    x = self.bn5(x)
    x = self.act6(x)
    skipConn3 = x

    x = self.conv6(x)
    x = self.bn6(x)
    x = self.act7(x)
    x = torch.add(x, skipConn3)

    x = self.conv7(x)
    x = self.bn7(x)
    x = self.act8(x)
    skipConn4 = x

    x = self.conv8(x)
    x = self.bn8(x)
    x = self.act9(x)
    x = torch.add(skipConn4, x)

    x = self.pool2(x)
    x = self.act10(x)
    x = x.view(x.size(0), x.size(1)*x.size(2)*x.size(3))
    x = self.fc1(x)
    x = self.act11(x)
    x = self.fc2(x)
    x = self.act12(x)
    x = self.fc3(x)
    return x

In [13]:
batch_size = 2640
loss = torch.nn.CrossEntropyLoss()
lenet = Lenet2().to(device)
optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
test_accuracy_history = []

for epoch in bar(range(200), desc = 'learning'):
  order = np.random.permutation(len(train_data))

  for start_index in range(0, len(train_data), batch_size):
    optimizer.zero_grad()

    batch_indexes = order[start_index:start_index+batch_size]

    data_batch = train_data[batch_indexes].to(device)
    label_batch = train_labels[batch_indexes].to(device)

    preds = lenet.forward(data_batch)

    loss_val = loss(preds, label_batch)
    loss_val.backward()

    optimizer.step()

  with torch.no_grad():
    test_preds = lenet.forward(test_data)
    test_accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
    test_accuracy_history.append(test_accuracy)

<ipython-input-13-72e9c02e7df9>:7: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



learning:   0%|          | 0/200 [00:00<?, ?it/s]

In [15]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history, mode ="lines+markers"))
fig.update_layout(title="accuracy default model", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#85

# Third net

In [17]:
class Lenet3(torch.nn.Module):
  def __init__(self, dp_rate):
    super(Lenet3, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
    self.act1 = torch.nn.ReLU6()
    self.dropout1 = torch.nn.Dropout(p=dp_rate)
    self.conv2 = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, padding=1)
    self.act2 = torch.nn.ReLU6()
    self.dropout2 = torch.nn.Dropout(p=dp_rate)
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=6, kernel_size=3, padding=1)
    self.act3 = torch.nn.ReLU6()
    self.dropout3 = torch.nn.Dropout(p=dp_rate)
    self.conv4 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
    self.act4 = torch.nn.ReLU6()
    self.dropout4 = torch.nn.Dropout(p=dp_rate)
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act5 = torch.nn.ReLU6()
    self.conv5 = torch.nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=0)
    self.act6 = torch.nn.ReLU6()
    self.dropout5 = torch.nn.Dropout(p=dp_rate)
    self.conv6 = torch.nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, padding=1)
    self.act7 = torch.nn.ReLU6()
    self.dropout6 = torch.nn.Dropout(p=dp_rate)
    self.conv7 = torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3, padding=0)
    self.act8 = torch.nn.ReLU6()
    self.dropout7 = torch.nn.Dropout(p=dp_rate)
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
    self.act9 = torch.nn.ReLU6()
    self.dropout8 = torch.nn.Dropout(p=dp_rate)
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act10 = torch.nn.ReLU6()
    self.fc1 = torch.nn.Linear(16*5*5, 120)
    self.act11 = torch.nn.ReLU6()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act12 = torch.nn.ReLU6()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.act1(x)
    x = self.dropout1(x)
    skipConn1 = x

    x = self.conv2(x)
    x = self.act2(x)
    x = self.dropout2(x)
    x = torch.add(skipConn1, x)

    x = self.conv3(x)
    x = self.act3(x)
    x = self.dropout3(x)
    skipConn2 = x

    x = self.conv4(x)
    x = self.act4(x)
    x = self.dropout4(x)
    x = torch.add(skipConn2, x)

    x = self.pool1(x)
    x = self.act5(x)
    x = self.conv5(x)
    x = self.act6(x)
    x = self.dropout5(x)
    skipConn3 = x
    x = self.conv6(x)
    x = self.act7(x)
    x = self.dropout6(x)
    x = torch.add(x, skipConn3)
    x = self.conv7(x)
    x = self.act8(x)
    x = self.dropout7(x)
    skipConn4 = x

    x = self.conv8(x)
    x = self.act9(x)
    x = self.dropout8(x)
    x = torch.add(skipConn4, x)

    x = self.pool2(x)
    x = self.act10(x)
    x = x.view(x.size(0), x.size(1)*x.size(2)*x.size(3))
    x = self.fc1(x)
    x = self.act11(x)
    x = self.fc2(x)
    x = self.act12(x)
    x = self.fc3(x)
    return x

In [23]:
dp_rate = [0.5, 0.1, 0.05, 0.01, 0.005, 0.01]
batch_size = 2640
loss = torch.nn.CrossEntropyLoss()

test_accuracy_history = [[] for i in range(6)]

for rate in bar(range(6), desc='rate', leave=False):
  lenet = Lenet3(dp_rate[rate]).to(device)
  optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
  for epoch in bar(range(200), desc = 'learning', leave=False):
    order = np.random.permutation(len(train_data))

    for start_index in range(0, len(train_data), batch_size):
      optimizer.zero_grad()

      batch_indexes = order[start_index:start_index+batch_size]

      data_batch = train_data[batch_indexes].to(device)
      label_batch = train_labels[batch_indexes].to(device)

      preds = lenet.forward(data_batch)

      loss_val = loss(preds, label_batch)
      loss_val.backward()

      optimizer.step()

    with torch.no_grad():
      test_preds = lenet.forward(test_data)
      test_accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
      test_accuracy_history[rate].append(test_accuracy)

<ipython-input-23-0659bdf6ea5f>:7: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



rate:   0%|          | 0/6 [00:00<?, ?it/s]

<ipython-input-23-0659bdf6ea5f>:10: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

In [24]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[0], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.5", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#190

In [25]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[1], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.1", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#110

In [26]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[2], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.05", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#120

In [27]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[3], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.01", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#100

In [28]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[4], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.005", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#80

In [29]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[5], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.01", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#100

# Four net

In [5]:
class Lenet4(torch.nn.Module):
  def __init__(self, dp_rate):
    super(Lenet4, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
    self.bn1 = torch.nn.BatchNorm2d(4)
    self.act1 = torch.nn.ReLU6()
    self.dropout1 = torch.nn.Dropout(p=dp_rate)
    self.conv2 = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, padding=1)
    self.bn2 = torch.nn.BatchNorm2d(4)
    self.act2 = torch.nn.ReLU6()
    self.dropout2 = torch.nn.Dropout(p=dp_rate)
    self.conv3 = torch.nn.Conv2d(in_channels=4, out_channels=6, kernel_size=3, padding=1)
    self.bn3 = torch.nn.BatchNorm2d(6)
    self.act3 = torch.nn.ReLU6()
    self.dropout3 = torch.nn.Dropout(p=dp_rate)
    self.conv4 = torch.nn.Conv2d(in_channels=6, out_channels=6, kernel_size=3, padding=1)
    self.bn4 = torch.nn.BatchNorm2d(6)
    self.act4 = torch.nn.ReLU6()
    self.dropout4 = torch.nn.Dropout(p=dp_rate)
    self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act5 = torch.nn.ReLU6()
    self.conv5 = torch.nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=0)
    self.bn5 = torch.nn.BatchNorm2d(12)
    self.act6 = torch.nn.ReLU6()
    self.dropout5 = torch.nn.Dropout(p=dp_rate)
    self.conv6 = torch.nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, padding=1)
    self.bn6 = torch.nn.BatchNorm2d(12)
    self.act7 = torch.nn.ReLU6()
    self.dropout6 = torch.nn.Dropout(p=dp_rate)
    self.conv7 = torch.nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3, padding=0)
    self.bn7 = torch.nn.BatchNorm2d(16)
    self.act8 = torch.nn.ReLU6()
    self.dropout7 = torch.nn.Dropout(p=dp_rate)
    self.conv8 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
    self.bn8 = torch.nn.BatchNorm2d(16)
    self.act9 = torch.nn.ReLU6()
    self.dropout8 = torch.nn.Dropout(p=dp_rate)
    self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
    self.act10 = torch.nn.ReLU6()
    self.fc1 = torch.nn.Linear(16*5*5, 120)
    self.act11 = torch.nn.ReLU6()
    self.fc2 = torch.nn.Linear(120, 84)
    self.act12 = torch.nn.ReLU6()
    self.fc3 = torch.nn.Linear(84, 35)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act1(x)
    x = self.dropout1(x)
    skipConn1 = x

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.act2(x)
    x = self.dropout2(x)
    x = torch.add(skipConn1, x)

    x = self.conv3(x)
    x = self.bn3(x)
    x = self.act3(x)
    x = self.dropout3(x)
    skipConn2 = x

    x = self.conv4(x)
    x = self.bn4(x)
    x = self.act4(x)
    x = self.dropout4(x)
    x = torch.add(skipConn2, x)

    x = self.pool1(x)
    x = self.act5(x)

    x = self.conv5(x)
    x = self.bn5(x)
    x = self.act6(x)
    x = self.dropout5(x)
    skipConn3 = x

    x = self.conv6(x)
    x = self.bn6(x)
    x = self.act7(x)
    x = self.dropout6(x)
    x = torch.add(x, skipConn3)

    x = self.conv7(x)
    x = self.bn7(x)
    x = self.act8(x)
    x = self.dropout7(x)
    skipConn4 = x

    x = self.conv8(x)
    x = self.bn8(x)
    x = self.act9(x)
    x = self.dropout8(x)
    x = torch.add(skipConn4, x)

    x = self.pool2(x)
    x = self.act10(x)
    x = x.view(x.size(0), x.size(1)*x.size(2)*x.size(3))
    x = self.fc1(x)
    x = self.act11(x)
    x = self.fc2(x)
    x = self.act12(x)
    x = self.fc3(x)
    return x

In [7]:
dp_rate = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
batch_size = 2640
loss = torch.nn.CrossEntropyLoss()

test_accuracy_history = [[] for i in range(6)]

for rate in bar(range(6), desc='rate', leave=False):
  lenet = Lenet4(dp_rate[rate]).to(device)
  optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
  for epoch in bar(range(200), desc = 'learning', leave=False):
    order = np.random.permutation(len(train_data))

    for start_index in range(0, len(train_data), batch_size):
      optimizer.zero_grad()

      batch_indexes = order[start_index:start_index+batch_size]

      data_batch = train_data[batch_indexes].to(device)
      label_batch = train_labels[batch_indexes].to(device)

      preds = lenet.forward(data_batch)

      loss_val = loss(preds, label_batch)
      loss_val.backward()

      optimizer.step()

    with torch.no_grad():
      test_preds = lenet.forward(test_data)
      test_accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
      test_accuracy_history[rate].append(test_accuracy)

<ipython-input-7-f6b6d00b5c9d>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for rate in bar(range(6), desc='rate', leave=False):


rate:   0%|          | 0/6 [00:00<?, ?it/s]

<ipython-input-7-f6b6d00b5c9d>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in bar(range(200), desc = 'learning', leave=False):


learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

learning:   0%|          | 0/200 [00:00<?, ?it/s]

In [10]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[0], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.5", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#110

In [11]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[1], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.1", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#130

In [12]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[2], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.05", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#120

In [13]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[3], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.01", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#70

In [14]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[4], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.005", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#100

In [15]:
fig = go.Figure(data=go.Scatter(x=[i for i in range(200)], y=test_accuracy_history[5], mode ="lines+markers"))
fig.update_layout(title="accuracy default model rate 0.001", xaxis_title = "epoch", yaxis_title = "accuracy")
fig.show()#120

#learning 1net

In [ ]:
accuracy_net = []
for experiment in bar(range(20), desc = 'sample eperimnet'):

  loss = torch.nn.CrossEntropyLoss()
  lenet = Lenet1().to(device)

  optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
  batch_size = 2640
  max_acc = 0

  for epoch in range(120):
    order = np.random.permutation(len(train_data))

    for start_index in range(0, len(train_data), batch_size):
      optimizer.zero_grad()

      batch_indexes = order[start_index:start_index+batch_size]

      data_batch = train_data[batch_indexes]
      labels_batch= train_labels[batch_indexes]

      preds = lenet.forward(data_batch)

      loss_val = loss(preds, labels_batch)
      loss_val.backward()

      optimizer.step()
    with torch.no_grad():
      test_preds = lenet.forward(test_data)
      accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
      max_acc = max(max_acc, accuracy)
  accuracy_net.append(max_acc)
  print(max_acc)
print('accuracy')
print(*accuracy_net)

#learning 2net

In [ ]:
accuracy_net = []
for experiment in bar(range(20), desc = 'sample eperimnet', leave = False):

  loss = torch.nn.CrossEntropyLoss()
  lenet = Lenet2().to(device)

  optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
  batch_size = 2640
  max_acc = 0

  for epoch in bar(range(85), leave=False):
    order = np.random.permutation(len(train_data))

    for start_index in range(0, len(train_data), batch_size):
      optimizer.zero_grad()

      batch_indexes = order[start_index:start_index+batch_size]

      data_batch = train_data[batch_indexes]
      labels_batch= train_labels[batch_indexes]

      preds = lenet.forward(data_batch)

      loss_val = loss(preds, labels_batch)
      loss_val.backward()

      optimizer.step()
    with torch.no_grad():
      test_preds = lenet.forward(test_data)
      accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
      max_acc = max(max_acc, accuracy)
  accuracy_net.append(max_acc)
  print(max_acc)
print('accuracy')
print(*accuracy_net)

#learning 3net

In [ ]:
dp_rate = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
epochs = [110,130,120,70,100,120]

for rate in bar(range(6), desc = "rate", leave= False)
  accuracy_net = []
  for experiment in bar(range(20), desc = 'sample eperimnet', leave = False):

    loss = torch.nn.CrossEntropyLoss()
    lenet = Lenet3(dp_rate[rate]).to(device)

    optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
    batch_size = 2640
    max_acc = 0

    for epoch in bar(range(epochs[rate]), leave=False):
      order = np.random.permutation(len(train_data))

      for start_index in range(0, len(train_data), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        data_batch = train_data[batch_indexes]
        labels_batch= train_labels[batch_indexes]

        preds = lenet.forward(data_batch)

        loss_val = loss(preds, labels_batch)
        loss_val.backward()

        optimizer.step()
      lenet.eval()
      with torch.no_grad():
        test_preds = lenet.forward(test_data)
        accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
        max_acc = max(max_acc, accuracy)
      lenet.train()
    accuracy_net.append(max_acc)
    print(max_acc)
  print('accuracy')
  print(*accuracy_net)

#learning 4net

In [ ]:
dp_rate = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
epochs = [110,130,120,70,100,120]

for rate in bar(range(6), desc = "rate", leave= False)
  accuracy_net = []
  for experiment in bar(range(20), desc = 'sample eperimnet', leave = False):

    loss = torch.nn.CrossEntropyLoss()
    lenet = Lenet4(dp_rate[rate]).to(device)

    optimizer = torch.optim.RMSprop(lenet.parameters(), lr=0.001)
    batch_size = 2640
    max_acc = 0

    for epoch in bar(range(epochs[rate]), leave=False):
      order = np.random.permutation(len(train_data))

      for start_index in range(0, len(train_data), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        data_batch = train_data[batch_indexes]
        labels_batch= train_labels[batch_indexes]

        preds = lenet.forward(data_batch)

        loss_val = loss(preds, labels_batch)
        loss_val.backward()

        optimizer.step()
      lenet.eval()
      with torch.no_grad():
        test_preds = lenet.forward(test_data)
        accuracy = (test_preds.argmax(dim=1)==test_labels).float().mean().data.cpu()
        max_acc = max(max_acc, accuracy)
      lenet.train()
    accuracy_net.append(max_acc)
    print(max_acc)
  print('accuracy')
  print(*accuracy_net)